# HMM Generation

Code to generate new pieces using variations of Hidden Markov Models

Models are designed to be trained on one piece at a time



In [1]:
## Scripts containing inference algorithms
from BaumWelch import *
from TVAR import *

In [2]:
import numpy as np
import pandas as pd
import csv
from numpy import linspace,exp
from numpy.random import randn
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
import seaborn as sns

import scipy 
import editdistance
import sklearn.metrics
import statsmodels.api as sm

%matplotlib inline 

In [3]:
## Helper functions for working with note pitch representation

## Convert from pitch representation (integers 0-127) to integers (0-max)
## x is the input vector of notes and code is a vector of the unique pitches in x
def encode(x, code):
    output = np.array([int(np.where(code == x[i])[0]) for i in range(0, len(x))])
    return output


## Reverses the function encode
## x is the vector of pitches to decode and code is a vector of the unique pitches in x before it was encoded
def decode(x, code):
    output = np.zeros(len(x))
    for i in range(0, len(x)):
        output[i] = code[x[i]]
    return output

## Function to convert the values in array to the nearest values in the array value
## Used to convert continues TVAR generated pitches to closest integer values for MIDI representation
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

## Generate New Pieces

Functions to take learned parameters and generate new pieces

In [4]:
## Function to generate new pieces from the HMM, 2-HMM, 3-HMM, LR-HMM, 2LR-HMM, and 3-LR HMM
## n is the length of the original and generated piece
## pi is the learned initial distribution
## phi is the learned emission distribution
## Tmat is the learned transition matrix
## T2mat is the learned second order transition matrix (if applicable)
## T3mat is the learned third order transition matrix (if applicable)
## code is the unique note pitches occurring in the original piece
## model is the model order which the input parameters correspond to, either "first_order", "second_order" or "third_order"
## Outputs: output is the note pitches of the generated pieces (array of length n), z are the generated hidden states 
##          (vector of length n)
def hmm(n, pi, phi, Tmat, T2mat, T3mat, code, model):
    m = Tmat.shape[0]
    k = phi.shape[1]
    zstates = np.arange(0, m, dtype = int)
    xstates = np.arange(0, k, dtype = int)
    z = np.zeros(n, dtype = int)
    x = np.zeros(n, dtype = int)
    z[0] = np.random.choice(zstates, size = 1, p = pi)
    if model == 'first_order':
        for j in range(1, n):
            z[j] = np.random.choice(zstates, size = 1, p = Tmat[z[j-1], :])
        for i in range(0, n):
            x[i] = np.random.choice(xstates, size = 1, p = phi[z[i], :])
     
    if model == 'second_order':
        z[1] = np.random.choice(zstates, size = 1,  p = Tmat[z[0], :])
        for j in range(2, n):
            z[j] = np.random.choice(zstates, size = 1,  p = T2mat[z[j-2],z[j-1], :])
        for i in range(0, n):
            x[i] = np.random.choice(xstates, size =1, p = phi[z[i], :])
    if model == 'third_order':
        z[1] = np.random.choice(zstates, size = 1,  p = Tmat[z[0], :])
        z[2] = np.random.choice(zstates, size = 1,  p = T2mat[z[0],z[1], :])
        for j in range(3, n):
            z[j] = np.random.choice(zstates, size = 1,  p = T3mat[z[j-3],z[j-2],z[j-1], :])
        for i in range(0, n):
            x[i] = np.random.choice(xstates, size =1, p = phi[z[i], :])
    output = decode(x, code)
    return (output, z)



# Pre-Processing

In [5]:
## Function to pre-process input CSV of original song into form that can be used for modeling and metrics
## Assumes original piece's MIDI file has been converted to a CSV using http://www.fourmilab.ch/webtools/midicsv/#midicsv.5
## input_filename = name of original csv
## output_filename = name of new csv to save generated piece to
## Outputs:
##         quarter_note = number of time steps corresponding to 1 quarter note
##         num = numerator in key signature
##         denom = denominator in key signature
##         key = key signature of piece, integer between -7 and 7 where 0 is C Major
##         measures = number of measures in input piece
##         time = vector of time stamps at which notes occurr
##         notes = vector of note pitches (integers 0-127)
##         velocity = "volume" of each note pitch, 0 = note off, length of time is the same as length of notes and velocity
##         song = pandas dataframe to use for output generated pieces, retains formatting expected by MIDI-CSV
##         song.index = index of original song dataframe

## See http://www.fourmilab.ch/webtools/midicsv/#midicsv.5 for a discussion of MIDI and CSV formats

class pre_process(object):
    def __init__(self, input_filename):
        self.input_filename = input_filename
        
    
    def read_process(self):
        with open(self.input_filename,encoding = "ISO-8859-1") as fd:
            reader=csv.reader(fd)
            rows= [row for idx, row in enumerate(reader)]
        song = pd.DataFrame(rows)
        r,c = np.where(song == ' Header')
        quarter_note = song.iloc[r,5].values.astype(int)[0]
        r, c = np.where(song == ' Time_signature')
        num = song.iloc[r, 3].values.astype(int)[0]
        denom = song.iloc[r, 4].values.astype(int)[0]**2
        try:
            r, c = np.where(song == ' Key_signature')
            key = song.iloc[r,3].values.astype(int)[0]
        except:
            key = None
        
        song_model = song.loc[song.iloc[:,0] == np.max(song.iloc[:,0])]
        song_model = song_model[song_model.iloc[:, 2].isin([' Note_on_c', ' Note_off_c'])]
        time = np.array(song_model.iloc[:,1]).astype(int)
        notes = np.array(song_model.iloc[:,4]).astype(int)
        velocity = np.array(song_model.iloc[:,5]).astype(int)
#         measures = np.round(np.max(time)/quarter_note)/num
#         min_note = quarter_note
#         actual = np.arange(0, min_note*measures*num, min_note).astype(int) 
#         time = np.array([find_nearest(actual, time[i]) for i in range(len(time))]).astype(int)
        return(time, notes, velocity, song, song_model.index)

## Velocity

Velocity is not explicitly modeled in this work.  Velocity is the "loudness" of each note pitch, where 0 is the note is off.  The first occurrence of a note pitch turns a note on and the next occurrence turns the same pitch off.  After a new sequence of note pitches was generated, the velocity for turning the pitch on and off was imputed accordingly.  The remaining non-0 velocities were produced using splines of the velocities of the original piece.

In [6]:
## newNotes = vector of note pitches of new, generated piece
## velocity = velocity of original piece
## newVelocities = velocities for newNotes, with 0s appropriately filled in and spline values for other non-0 values

def find_vel(newNotes, velocity):
    # Use splines to interpolate the velocities
    newVelocities = np.zeros(len(newNotes))
    y = velocity[np.nonzero(velocity)]
    indicies = []
    for i in np.unique(newNotes):
        indicies.append(np.where(newNotes == i)[0][::2])  ## set every other pitch occurrence to 0 (turn off)

    unlist = [item for sublist in indicies for item in sublist]
    unlist.sort()
    X = np.array(range(0,len(y)))
    s = UnivariateSpline(X, y, s=300) #750
    xs = np.linspace(0, len(y), len(unlist), endpoint = True)
    ys = s(xs)    
    newVelocities[np.array(unlist)] = np.round(ys).astype(int)
    #Fix entries that are too small or too large due to spline overfitting
    newVelocities[np.where(newVelocities < 0)[0]] = y[-1]
    newVelocities = newVelocities.astype(int)    

    return(newVelocities)

# HMM_compose

This is the main function to take in an original piece, learn the appropriate model parameters, generate a new piece, calculate metrics and output the results.
**Inputs:**
- input_filename = csv file of original piece (converted from MIDI using http://www.fourmilab.ch/webtools/midicsv/#midicsv.5)
- output_filename = filename for csvs of generated pieces and metrics
- min_note = length of shortest note occurring in original piece
- model = appropriate HMM model to fit, options include 'first_order', 'second_order', 'third_order', 'TVAR', 'factorial' and 'layered'
- m = number of hidden states for model 
- tol = tolerance for convergence of inference algorithms


**Outputs:**
- generated piece to 'output_filename', if multiple pieces are generated for metrics, last generated piece is saved by default
- metrics for number of generated pieces specified by it are saved to metrics folder
- If case_study = True:
    - time = time stamp for each note in the original and generated pieces
    - notes = original notes
    - newNotes = generated note pitches
    - z = sequence of generated hidden states
    - pi1 = learned initial distribution
    - phi1 = learned emission distribution
    - Tmat1 = learned transition distribution

Note: printed intergers correspond to iteration of inference algorithm

In [25]:
def hmm_compose(input_filename, output_filename, model, m,  tol = 0.01):
    time, notes, velocity, song, ind = pre_process(input_filename).read_process()

    #Find possible unique notes and velocities
    possibleNotes = np.unique(notes)
    possibleVelocities =  np.unique(velocity)

    k = len(possibleNotes)
    xNotes = encode(notes, possibleNotes)
    n = len(xNotes)
    


    #Run BaumWelch for specified model
    if model == 'first_order':
        it1, p1, pi1, phi1, Tmat1 = first_order(n, m, k, xNotes, tol)
        newNotes, z  = hmm(n, pi1, phi1, Tmat1, None, None, possibleNotes,'first_order')
        newVelocities = find_vel(newNotes, velocity)
         
                  
    if model == 'second_order':
        it1, p1, pi1, phi1, Tmat1 = first_order(n, m, k, xNotes, tol)
        it2, T2mat = second_order(n, m, k, xNotes, pi1, Tmat1, phi1, tol)
        newNotes,z  = hmm(n, pi1, phi1, Tmat1, T2mat, None, possibleNotes,'second_order')
        newVelocities = find_vel(newNotes, velocity)
        
        

    if model == 'third_order':
        it1, p1, pi1, phi1, Tmat1 = first_order(n, m, k, xNotes, tol)
        it2, T2mat = second_order(n, m, k, xNotes, pi1, Tmat1, phi1, tol)
        it3, T3mat = third_order(n, m, k, xNotes, pi1, Tmat1, T2mat, phi1, tol)
        newNotes,z  = hmm(n, pi1, phi1, Tmat1, T2mat, T3mat, possibleNotes,'third_order')
        newVelocities = find_vel(newNotes, velocity)
        

    if model == 'TVAR':
        # Find parameters that maximize likelihood
        x = notes - np.mean(notes)
        T = n
        pvals=np.array([7, 15]) 
        p=pvals[1]  
        dn=np.arange(0.94, 0.975,.005) 
        bn=np.arange(0.85, 0.915, 0.005) 
        m0=np.zeros(shape = (p,1)); n0=1; s0=0.01; C0=np.identity(p); 
        [popt,delopt,likp] = tvar_lik(x,pvals,dn,bn,m0,C0,s0,n0);
        print(popt)

        # Fit TVAR
        p=popt; m0=np.zeros(shape = (p,1)); n0=1; s0=0.01; C0=np.identity(p);  # initial priors 
        delta=delopt
        [m,C,n,s,e,mf,Cf,sf,nf,ef,qf] = tvar(x,p,delta,m0,C0,s0,n0);

        # Simulate from TVAR
        N=100; # MC sample size
        times = np.arange(T);
        phis = tvar_sim(m,C,n,s,times,N)
        print(phis.shape)

        # Generate new notes

        err_term = np.random.normal(0, np.sqrt(s))
        z = np.zeros(len(notes))
        newNotes = x
        
        for t in range(p, T):
            if t == p:
                newNotes[t] = np.dot(x[t-1::-1], phis[:,t,0]) + err_term[t]
                z[t] = np.dot(x[t-1::-1], phis[:,t,0])

            else:
                newNotes[t] = np.dot(x[t-1:t-p-1:-1], phis[:,t,0]) + err_term[t]
                z[t] = np.dot(x[t-1:t-p-1:-1], phis[:,t,0])

        newNotes = np.round(newNotes + np.mean(notes))    


        for j in range(len(notes)):
            if newNotes[j] not in possibleNotes:
                newNotes[j] = find_nearest(possibleNotes, newNotes[j])
        newVelocities = find_vel(newNotes, velocity)


        m = popt

    if model == 'factorial':  #originally 15, 10, 5, but 5,5,5 for case_study
        xstates = range(0, k)
        noteArray = np.zeros(shape = (3, n))
        
        it1, p1, pi1, phi15, Tmat1 = first_order(n, 15, k, xNotes, tol)
        zStar15 = Viterbi(n, 15, k, np.log(pi1), np.log(Tmat1), np.log(phi15), xNotes)
        zStar15 = np.array(zStar15).astype(int)
        it1, p1, pi1, phi10, Tmat1 = first_order(n, 10, k, xNotes, tol)
        zStar10 = Viterbi(n, 10, k, np.log(pi1), np.log(Tmat1), np.log(phi10), xNotes)
        zStar10 = np.array(zStar10).astype(int)
        it1, p1, pi1, phi5, Tmat1 = first_order(n, 5, k, xNotes, tol)
        zStar5 = Viterbi(n, 5, k, np.log(pi1), np.log(Tmat1), np.log(phi5), xNotes)
        zStar5 = np.array(zStar5).astype(int)
        z = [zStar15, zStar10, zStar5]
        phi1 = [phi15, phi10, phi5]

       
        for j in range(0, n):
            noteArray[0,j] = np.random.choice(xstates, size = 1, p = phi15[zStar15[j], :])
            noteArray[1,j] = np.random.choice(xstates, size = 1, p = phi10[zStar10[j], :])
            noteArray[2,j] = np.random.choice(xstates, size = 1, p = phi5[zStar5[j], :])
        temp_notes = np.rint(np.mean(noteArray, axis=0)).astype(int)
        temp_notes = decode(temp_notes, possibleNotes)
        newNotes = temp_notes
        newVelocities = find_vel(newNotes, velocity)
        

    if model == 'layered':
        it1, p1, pi1, phi1, Tmat1 = first_order(n, m, k, xNotes, tol)
        zStar1 = Viterbi(n, m, k, np.log(pi1), np.log(Tmat1), np.log(phi1), xNotes)
        zStar1 = np.array(zStar1).astype(int)
        it2, p2, pi2, phi2, Tmat2 = first_order(n, m, m, zStar1, tol)
        zStar2 = Viterbi(n, m, m, np.log(pi2), np.log(Tmat2), np.log(phi2), zStar1)
        zStar2 = np.array(zStar2).astype(int)
        it3, p3, pi3, phi3, Tmat3 = first_order(n, m, m, zStar2, tol)
        zStar3 = Viterbi(n, m, m, np.log(pi3), np.log(Tmat3), np.log(phi3), zStar2)
        zStar3 = np.array(zStar3).astype(int)
        output = np.zeros(shape = (3,n), dtype = int)
        z = [zStar1, zStar2, zStar3]
        
        xstates = range(0, k)
        zstates = range(0, m)
        
        for j in range(0,n):
            output[2, j] = np.random.choice(zstates, size = 1, p = phi3[zStar3[j], :])
            output[1, j] = np.random.choice(zstates, size = 1, p = phi2[output[2, j], :])
            output[0, j] = np.random.choice(xstates, size = 1, p = phi1[output[1, j], :])
        temp_notes = decode(output[0,:], possibleNotes).astype(int)
        newNotes = temp_notes
        newVelocities = find_vel(newNotes, velocity)
        phi1 = [phi1, phi2, phi3]
        

    song.iloc[ind, 1] = time
    song.iloc[ind, 4] = newNotes
    song.iloc[ind, 5] = newVelocities
    song.iloc[ind[np.where(newVelocities !=0)], 2] = ' Note_on_c'
    song.iloc[ind[np.where(newVelocities ==0)], 2] = ' Note_off_c'
    split = output_filename.split('.')
    output_filename = split[0] + '__'+ model + '_' + str(m)+  '-tol' +str(tol)+'.' + split[1]
    
    song.to_csv(output_filename, header = None, index = False)
    

## First Order HMM - Example

In [10]:
np.random.seed(77)
hmm_compose('OriginalCSV/twinkle-twinkle-little-star.csv', 
                                                   'NewCSV/twinkle-twinkle-little-star.csv',  
                                                   'first_order', 5, 10E-3)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


# Beethoven

- Change model from first order
- Can also change number of hidden states (first number) and tolerance (second number)
- Increasing number of hidden states will take longer to run code
- Decreasing tolerance will make code converge faster

## Beethoven No3-01

In [11]:
hmm_compose('OriginalCSV/BeetNo3-01-bottom.csv','NewCSV/BeetNo3-01-bottom.csv', 'layered', 5, 10E-3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in log


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:121: RuntimeWarning: divide by zero encountered in log


In [12]:
hmm_compose('OriginalCSV/BeetNo3-01-top.csv','NewCSV/BeetNo3-01-top.csv', 'layered', 5, 10E-3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in log


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:118: RuntimeWarning: divide by zero encountered in log


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:121: RuntimeWarning: divide by zero encountered in log


## Beethoven No5-01

In [13]:
hmm_compose('OriginalCSV/BeetNo5-01-bottom.csv','NewCSV/BeetNo5-01-bottom.csv', 'layered', 5, 10E-3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
1
2
3
4
5

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in log



6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:118: RuntimeWarning: divide by zero encountered in log


1
2
3
4
5
6
7
8
9


In [14]:
hmm_compose('OriginalCSV/BeetNo5-01-top.csv','NewCSV/BeetNo5-01-top.csv', 'layered', 5, 10E-3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
1


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in log


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:118: RuntimeWarning: divide by zero encountered in log


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:121: RuntimeWarning: divide by zero encountered in log


## Beethoven No7-02

In [15]:
hmm_compose('OriginalCSV/BeetNo7-02-bottom.csv','NewCSV/BeetNo7-02-bottom.csv', 'layered', 5, 10E-3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in log


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:118: RuntimeWarning: divide by zero encountered in log


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:121: RuntimeWarning: divide by zero encountered in log


In [16]:
hmm_compose('OriginalCSV/BeetNo7-02-top.csv','NewCSV/BeetNo7-02-top.csv', 'layered', 5, 10E-3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in log


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:118: RuntimeWarning: divide by zero encountered in log


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:121: RuntimeWarning: divide by zero encountered in log


In [26]:
hmm_compose('OriginalCSV/BeetNo7-02-bottom.csv','NewCSV/BeetNo7-02-bottom.csv', 'TVAR', 5, 10E-3)

9
(9, 362, 100)


In [27]:
hmm_compose('OriginalCSV/BeetNo7-02-top.csv','NewCSV/BeetNo7-02-top.csv', 'TVAR', 5, 10E-3)

7
(7, 353, 100)


## Beethoven No9-04 - Ode to Joy Solo

In [28]:
hmm_compose('OriginalCSV/beethoven-symphony9-4-ode-to-joy-piano-solo.csv',
            'NewCSV/ode-to-joy.csv', 'layered', 5, 10E-3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in log


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:118: RuntimeWarning: divide by zero encountered in log


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:121: RuntimeWarning: divide by zero encountered in log


In [29]:
hmm_compose('OriginalCSV/beethoven-symphony9-4-ode-to-joy-piano-solo.csv',
            'NewCSV/ode-to-joy.csv', 'TVAR', 5, 10E-3)

8
(8, 532, 100)
